In [ ]:
import pandas as pd
import numpy as np
import regex as re

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep
from random import randint
from pandas import json_normalize

In [ ]:
# Importing csv

In [ ]:
top_songs = pd.read_csv('top_songs.csv')

In [ ]:
top_songs

In [ ]:
playlist = pd.read_csv('df_playlist.csv')
playlist

In [ ]:
X_playlist = playlist.select_dtypes(include='number')

In [ ]:
# KMeans 

In [ ]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(X_playlist)
X_prep=transformer.transform(X_playlist)
# Now, all features will have the same weight.
pd.DataFrame(X_prep,columns=X_playlist.columns).head()

In [ ]:
# Chosing K

In [ ]:
K = range(2, 20)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_prep)
    inertia.append(kmeans.inertia_)

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
from sklearn.metrics import silhouette_score
K = range(2, 20)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_prep)
    silhouette.append(silhouette_score(X_prep, kmeans.predict(X_prep)))


plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Slhouette score showing the optimal k')

In [ ]:
kmeans = KMeans(n_clusters=17, random_state=1234)
kmeans.fit(X_prep)

In [ ]:
# Predicting / assigning the clusters:
clusters = kmeans.predict(X_prep)

# Check the size of the clusters
pd.Series(clusters).value_counts().sort_index()

In [ ]:
# Saving my model

In [ ]:
pickle.dump(kmeans, open('kmeans.pkl', 'wb'))

In [ ]:
pickle.dump(transformer, open('X_prep.pkl', 'wb'))

In [ ]:
# obtaining song's cluster number
playlist['cluster']= clusters
playlist

In [ ]:
playlist_df = playlist[['track.name', 'name', 'cluster']]
playlist_df

In [ ]:
playlist_df.to_csv('playlist_df.csv', index=False)

In [ ]:
clusters

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(X_prep[:, 0], X_prep[:, 3], c=clusters,alpha=0.5)
plt.show()

In [ ]:
# Authentification Spotify API

In [ ]:
secrets_file = open("secrets.txt","r")

In [ ]:
string = secrets_file.read()

In [ ]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [ ]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [ ]:
def remove_special_characters(text):
    # Remove special characters using a regular expression
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def recommend_song(input_song, input_artist):

    # Convert the user input and song names to lowercase for case-insensitive comparison
    input_song_lower = input_song.lower()
    input_artist_lower = input_artist.lower()

    top_songs_lower = top_songs.apply(lambda x: x.str.lower().apply(remove_special_characters))
    top_songs_lower['song'] = top_songs_lower['song'].apply(remove_special_characters)

    # Check if the user-inputted song is in the top songs list (case-insensitive)
    if input_song_lower in top_songs_lower['song'].values:
        # Filter out the user-inputted song from the DataFrame
        top_songs_without_input = top_songs[
            top_songs_lower['song'] != input_song_lower
        ]

        # Recommend a random song from the filtered df
        recommended_song = top_songs_without_input.sample().squeeze()

        print(f"You might also like: {recommended_song['song']} by {recommended_song['artist']}")
        
    else:
        
        #Opening the spotify connection
        sp = open_spotify_connection()

        # Search for the user-inputted song and artist on Spotify
        search_results = sp.search(q=f"{input_song} {input_artist}", type='track', limit =1 )

        if not search_results['tracks']['items']:
            print("Song not found on Spotify. Please try another song.")
            return

        # Get the track ID for the user-inputted song
        if search_results['tracks']['total']!=0 :
            track_id = search_results['tracks']['items'][0]['id']
            #print(track_id)
        
        # Getting the audio features using the id
        new_data = sp.audio_features(track_id)
        #print(type(new_data[0]))
        data_df = json_normalize(new_data[0]) #normalizing the data to a df
        num_data_df = data_df.select_dtypes(include='number') #getting the numerical data only to fit
        #print(num_data_df)

        #Using pickle to get the existing scaler and kmean
        kmeans = pickle.load(open('kmean.pkl', 'rb'))
        scaler = pickle.load(open('scaler.pkl','rb'))
        X_prep = scaler.transform(num_data_df)
        predicted_cluster = kmeans.predict(X_prep)
        
        # Predict the cluster for the input song using the clustering model
        cluster = predicted_cluster[0]
        #print(cluster)
        
        # Filter songs in the same cluster from the playlist DataFrame and
        # Recommend a random song from the same cluster
        recommended_song = playlist_df[playlist_df['cluster']== cluster].sample().squeeze()
        # print(recommended_song)

        print(f"You might also like: {recommended_song['track.name']} by {recommended_song['name']}")

        # Introduce a respectful nap to avoid hitting rate limits
        sleep(randint(1, 3000) / 1000)


In [ ]:
def open_spotify_connection():
    #Opens a new spotify connection
    secrets_file = open("secrets.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    return sp;

In [ ]:
# Get user input for song and artist
input_song = input("Enter a song: ")
input_artist = input("Enter the artist: ")
recommend_song(input_song, input_artist )